In [115]:
# import libraries
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import os
import zxingcpp

In [116]:
# Create a QRCodeDetector object for OpenCV
qr_detector_opencv = cv2.QRCodeDetector()

# Get a list of all image files in the folder
folder_path = "mainImageFile/"
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Define the known lowest QR code position in pixels
lowest_qr_pos = 100

# Define the pixel to millimeter ratio
pixel_to_mm_ratio = 5

# Initialize the minimum number
min_number = 9999

# Initialize the minimum distance to water
min_dist_to_water = 9999

# Initialize lists to store numbers read from Pyzbar and Zxing
list_Number_Pyzbar = []
list_Number_Zxing = []


In [117]:
# Loop over each file
for image_file in image_files:
    # Load the image
    image = cv2.imread(os.path.join(folder_path, image_file))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the image using Otsu's method
    ret, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Check the distance between the lowest QR code and the water
    water_pos = np.where(threshold[lowest_qr_pos:] == 0)[0]
    if len(water_pos) > 0:
        dist_to_water = water_pos[0] / pixel_to_mm_ratio
        if dist_to_water < min_dist_to_water:
            min_dist_to_water = dist_to_water

    # Use Pyzbar to decode QR codes in the image
    decoded_data_pyzbar = decode(threshold)
    if decoded_data_pyzbar:
        # Loop over each QR code in the image
        for qrCode in decoded_data_pyzbar:
            # Get the number from the QR code
            number = int(qrCode.data)
            list_Number_Pyzbar.append(number)

            # Update the minimum number if necessary
            if number < min_number:
                min_number = number

    # Use Zxing to decode QR codes in the image
    results_zxing = zxingcpp.read_barcodes(threshold)
    for result in results_zxing:
        number = int(result.text)
        list_Number_Zxing.append(number)
        if number < min_number:
            min_number = number

In [119]:
# Check if any QR code was successfully read
if min_number == 9999:
    print("Could not read any QR codes from the photos.")
else:
    # Print the minimum number from all QR codes
    print("Minimum number from all QR codes:", min_number)
    # Print the numbers read from Pyzbar
    print('Pyzbar:', list_Number_Pyzbar)
    # Print the numbers read from Zxing
    print('Zxing:', list_Number_Zxing)
    # Print the Minimum distance to water
    print("Minimum distance to water:", min_dist_to_water, "mm")
    # Print result
    print("Water Level: ", min_number, "cm", min_dist_to_water, "mm")

Minimum number from all QR codes: 2
Pyzbar: [17, 11, 5, 3, 2, 6]
Zxing: [17, 25, 11, 5, 7, 3, 2, 4, 6, 13, 29]
Minimum distance to water: 0.0 mm
Water Level:  2 cm 0.0 mm
